In [1]:
from glob import glob
import numpy as np
from itertools import combinations
from polyphys.manage import organizer
from polyphys.manage import utilizer 
from polyphys.manage.parser import HnsCyl, HnsCub
from polyphys.analyze import clusters
from polyphys.analyze import measurer
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import defaultdict


# Test

In [ ]:
D=np.array([[
0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 ],[
0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0
]], dtype=np.int_)
M=np.array([[
0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 ],[
1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],[
0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 , 1 , 0 , 0 , 0 , 0 ]],dtype=np.int_)
print(M)

In [ ]:
M_created = clusters.generate_mon_bind_direct(D,2)
print(M_created)
print(M_created.shape)
print(np.sum(M_created==M))

In [ ]:
M_created_dangler, M_created_bridger = clusters.split_binder_matrix(M_created)

In [ ]:
M_created.shape

In [ ]:
M_created_bridger.shape

In [ ]:
M_created_dangler.shape

In [ ]:
bind_bind_dir_mat = clusters.find_binder_clusters(M_created)
print(bind_bind_dir_mat)
print()
bind_bind_full_mat = \
    clusters.generate_contact_matrix(bind_bind_dir_mat)
print(bind_bind_full_mat)
bind_clusters = clusters.count_foci_clusters(bind_bind_full_mat)
print(bind_clusters)
bind_contacts = clusters.count_foci_bonds(bind_bind_dir_mat)
print(bind_contacts)

# Workding with real files

## From Mon-HNS-Patch distance matrix

In [ ]:
# Pre-allocate some space
database = '/Users/amirhsi_mini/research_data/hns_distance/'
wholes = organizer.sort_filenames(glob(database+"N*.npy"),fmts=['npy'])
for idx in range(len(wholes)):
    whole = wholes[idx][0]
    print(whole)
    dist_m_hp = np.load(whole)
    sim_info = HnsCub(
            whole,
            'whole',
            'cubic',
            'nucleoid',
            'ring'
        )
    sim_name = sim_info.lineage_name + "-" + sim_info.group
    lj_cut = 2**(1/6)
    r_cutoff = np.round(
        0.5 * lj_cut * (sim_info.dmon + sim_info.dhns_patch), 3
        )
    n_frames , n_mon, n_hpatch = dist_m_hp.shape
    n_hcore = n_hpatch // 2
    # Process each time step
    foci_t = np.zeros((n_frames, n_hcore, n_hcore))
    dir_contacts_t = np.zeros(
        (n_frames, n_hcore, n_hcore), dtype=int
    )
    bonds_t = np.zeros((n_frames, n_hcore), dtype=int)
    clusters_t = np.zeros((n_frames, n_hcore+1), dtype=int)

    bonds_dangler_t = np.zeros((n_frames, n_hcore), dtype=int)
    clusters_dangler_t = np.zeros((n_frames, n_hcore+1), dtype=int)

    bonds_bridger_t = np.zeros((n_frames, n_hcore), dtype=int)
    clusters_bridger_t = np.zeros((n_frames, n_hcore+1), dtype=int)
    save_to = "/Users/amirhsi_mini/research_data/"
    for idx in range(n_frames):
        dir_m_hp_dirty = clusters.find_direct_contacts(
            dist_m_hp[idx], r_cutoff, inclusive=False
            )
        dir_m_hp = clusters.enforce_single_patch_dir_contact(dir_m_hp_dirty)
        dir_m_hc = clusters.generate_mon_bind_direct(dir_m_hp,2)
        dir_m_hc_dangler, dir_m_hc_bridger = clusters.split_binder_matrix(dir_m_hc)
        
        # bound binder direct contact matirx
        dir_hc_hc = clusters.find_binder_clusters(dir_m_hc)
        bonds_stat = clusters.count_foci_bonds(dir_hc_hc)
        bonds_t[idx] = bonds_stat 
        full_hc_hc = clusters.generate_contact_matrix(dir_hc_hc)
        clusters_stat = clusters.count_foci_clusters(full_hc_hc)
        clusters_t[idx] = clusters_stat
        # dangling binder direct contact matirx
        dir_hc_hc = clusters.find_binder_clusters(dir_m_hc_dangler)
        bonds_stat = clusters.count_foci_bonds(dir_hc_hc)
        bonds_dangler_t[idx] = bonds_stat
        full_hc_hc = clusters.generate_contact_matrix(dir_hc_hc)
        clusters_stat = clusters.count_foci_clusters(full_hc_hc)
        clusters_dangler_t[idx] = clusters_stat
        # bridging binder direct contact matirx
        dir_hc_hc = clusters.find_binder_clusters(dir_m_hc_bridger) 
        bonds_stat = clusters.count_foci_bonds(dir_hc_hc)
        bonds_bridger_t[idx] = bonds_stat
        full_hc_hc = clusters.generate_contact_matrix(dir_hc_hc)
        clusters_stat = clusters.count_foci_clusters(full_hc_hc)
        clusters_bridger_t[idx] = clusters_stat
    
    np.save(save_to + sim_name + '-bondsHistTHnsCore.npy', bonds_t)
    np.save(save_to + sim_name + '-clustersHistTHnsCore.npy', clusters_t)
    np.save(save_to + sim_name + '-bondsHistDangleTHnsCore.npy', bonds_dangler_t)
    np.save(save_to + sim_name + '-clustersHistDangleTHnsCore.npy', clusters_dangler_t)
    np.save(save_to + sim_name + '-bondsHistBridgeTHnsCore.npy', bonds_bridger_t)
    np.save(save_to + sim_name + '-clustersHistBridgeTHnsCore.npy', clusters_bridger_t)


In [ ]:
import traceback       
        
# Pre-allocate some space
database = '/Users/amirhsi_mini/research_data/hns_distance/'
wholes = organizer.sort_filenames(glob(database+"N*.npy"),fmts=['npy'])
for idx in range(len(wholes)):
    whole = wholes[idx][0]
    print(whole)
    dist_m_hp = np.load(whole)
    sim_info = HnsCub(
            whole,
            'whole',
            'cubic',
            'nucleoid',
            'ring'
        )
    sim_name = sim_info.lineage_name + "-" + sim_info.group
    lj_cut = 2**(1/6)
    r_cutoff = np.round(
        0.5 * lj_cut * (sim_info.dmon + sim_info.dhns_patch), 3
        )
    n_frames , n_mon, n_hpatch = dist_m_hp.shape
    n_hcore = n_hpatch // 2
    # Process each time step
    foci_t = np.zeros((n_frames, n_hcore, n_hcore))
    dir_contacts_t = np.zeros(
        (n_frames, n_hcore, n_hcore), dtype=int
    )
    bonds_t = np.zeros((n_frames, n_hcore), dtype=int)
    clusters_t = np.zeros((n_frames, n_hcore+1), dtype=int)

    bonds_dangler_t = np.zeros((n_frames, n_hcore), dtype=int)
    clusters_dangler_t = np.zeros((n_frames, n_hcore+1), dtype=int)

    bonds_bridger_t = np.zeros((n_frames, n_hcore), dtype=int)
    clusters_bridger_t = np.zeros((n_frames, n_hcore+1), dtype=int)
    save_to = "/Users/amirhsi_mini/research_data/"
    error_logs = pd.DataFrame(columns=['Frame_Index', 'Error_Message'])
    for idx in range(n_frames):
        dir_m_hp_dirty = clusters.find_direct_contacts(
            dist_m_hp[idx], r_cutoff, inclusive=False
            )
        dir_m_hp = clusters.enforce_single_patch_dir_contact(dir_m_hp_dirty)
        dir_m_hc = clusters.generate_mon_bind_direct(dir_m_hp, 2)
        dir_m_hc_dangler, dir_m_hc_bridger = \
            clusters.split_binder_matrix(dir_m_hc)
        try:
            # bound binder direct contact matirx
            dir_hc_hc = clusters.find_binder_clusters(dir_m_hc)
            bonds_stat = clusters.count_foci_bonds(dir_hc_hc)
            bonds_t[idx] = bonds_stat
            full_hc_hc = clusters.generate_contact_matrix(dir_hc_hc)
            if not clusters.is_symmetric(full_hc_hc):
                print("bridge")
                print(idx)
                break
            clusters_stat = clusters.count_foci_clusters(full_hc_hc)
            clusters_t[idx] = clusters_stat
            # dangling binder direct contact matirx
            dir_hc_hc = clusters.find_binder_clusters(dir_m_hc_dangler)
            bonds_stat = clusters.count_foci_bonds(dir_hc_hc)
            bonds_dangler_t[idx] = bonds_stat
            full_hc_hc = clusters.generate_contact_matrix(dir_hc_hc)
            clusters_stat = clusters.count_foci_clusters(full_hc_hc)
            if not clusters.is_symmetric(full_hc_hc):
                print("dangle")
                print(idx)
                break
            clusters_dangler_t[idx] = clusters_stat
            # bridging binder direct contact matirx
            dir_hc_hc = clusters.find_binder_clusters(dir_m_hc_bridger)
            bonds_stat = clusters.count_foci_bonds(dir_hc_hc)
            bonds_bridger_t[idx] = bonds_stat
            full_hc_hc = clusters.generate_contact_matrix(dir_hc_hc)
            if not clusters.is_symmetric(full_hc_hc):
                print("bridge")
                print(idx)
                break
            clusters_stat = clusters.count_foci_clusters(full_hc_hc)
            clusters_bridger_t[idx] = clusters_stat
        except Exception as e:
            # Log the error with frame index
            error_message = \
                f"Error at frame {idx} in whole {idx}: {str(e)}\n" + \
                f"Traceback: {traceback.format_exc()}"
            print(error_message)
            #new_row = pd.DataFrame({'Frame_Index': [idx], 'Error_Message': [error_message]})
            #error_logs = pd.concat([error_logs, new_row], ignore_index=True)
            break
            #continue
    
    np.save(save_to + sim_name + '-bondsHistTHnsCore.npy', bonds_t)
    np.save(save_to + sim_name + '-clustersHistTHnsCore.npy', clusters_t)
    np.save(save_to + sim_name + '-bondsHistDangleTHnsCore.npy', bonds_dangler_t)
    np.save(save_to + sim_name + '-clustersHistDangleTHnsCore.npy', clusters_dangler_t)
    np.save(save_to + sim_name + '-bondsHistBridgeTHnsCore.npy', bonds_bridger_t)
    np.save(save_to + sim_name + '-clustersHistBridgeTHnsCore.npy', clusters_bridger_t)


In [ ]:
import traceback       
        
# Pre-allocate some space
database = '/Users/amirhsi_mini/research_data/hns_distance/'
wholes = organizer.sort_filenames(glob(database+"N*.npy"),fmts=['npy'])
for idx in range(len(wholes)):
    whole = wholes[idx][0]
    print(whole)
    dist_m_hp = np.load(whole)
    sim_info = HnsCub(
            whole,
            'whole',
            'cubic',
            'nucleoid',
            'ring'
        )
    sim_name = sim_info.lineage_name + "-" + sim_info.group
    lj_cut = 2**(1/6)
    r_cutoff = np.round(
        0.5 * lj_cut * (sim_info.dmon + sim_info.dhns_patch), 3
        )
    n_frames , n_mon, n_hpatch = dist_m_hp.shape
    n_hcore = n_hpatch // 2
    # Process each time step
    foci_t = np.zeros((n_frames, n_hcore, n_hcore))
    dir_contacts_t = np.zeros(
        (n_frames, n_hcore, n_hcore), dtype=int
    )
    bonds_t = np.zeros((n_frames, n_hcore), dtype=int)
    clusters_t = np.zeros((n_frames, n_hcore+1), dtype=int)

    bonds_dangler_t = np.zeros((n_frames, n_hcore), dtype=int)
    clusters_dangler_t = np.zeros((n_frames, n_hcore+1), dtype=int)

    bonds_bridger_t = np.zeros((n_frames, n_hcore), dtype=int)
    clusters_bridger_t = np.zeros((n_frames, n_hcore+1), dtype=int)
    save_to = "/Users/amirhsi_mini/research_data/"
    error_logs = pd.DataFrame(columns=['Frame_Index', 'Error_Message'])
    for idx in range(n_frames):
        dir_m_hp_dirty = clusters.find_direct_contacts(
            dist_m_hp[idx], r_cutoff, inclusive=False
            )
        dir_m_hp = clusters.enforce_single_patch_dir_contact(dir_m_hp_dirty)
        dir_m_hc = clusters.generate_mon_bind_direct(dir_m_hp, 2)
        dir_m_hc_dangler, dir_m_hc_bridger = \
            clusters.split_binder_matrix(dir_m_hc)
        try:
            # bound binder direct contact matirx
            dir_hc_hc = clusters.find_binder_clusters(dir_m_hc)
            bonds_stat = clusters.count_foci_bonds(dir_hc_hc)
            bonds_t[idx] = bonds_stat
            full_hc_hc = clusters.generate_contact_matrix(dir_hc_hc)
            if not clusters.is_symmetric(full_hc_hc):
                print("full")
                print(idx)
                break
            clusters_stat = clusters.count_foci_clusters(full_hc_hc)
            clusters_t[idx] = clusters_stat
        except Exception as e:
            # Log the error with frame index
            error_message = \
                f"Error at frame {idx} in whole {idx}: {str(e)}\n" + \
                f"Traceback: {traceback.format_exc()}"
            print(error_message)
            #new_row = pd.DataFrame({'Frame_Index': [idx], 'Error_Message': [error_message]})
            #error_logs = pd.concat([error_logs, new_row], ignore_index=True)
            break
            #continue
    
    np.save(save_to + sim_name + '-bondsHistTHnsCore.npy', bonds_t)
    np.save(save_to + sim_name + '-clustersHistTHnsCore.npy', clusters_t)
    np.save(save_to + sim_name + '-bondsHistDangleTHnsCore.npy', bonds_dangler_t)
    np.save(save_to + sim_name + '-clustersHistDangleTHnsCore.npy', clusters_dangler_t)
    np.save(save_to + sim_name + '-bondsHistBridgeTHnsCore.npy', bonds_bridger_t)
    np.save(save_to + sim_name + '-clustersHistBridgeTHnsCore.npy', clusters_bridger_t)


In [ ]:
idx = 1251
dir_m_hp_dirty = clusters.find_direct_contacts(
        dist_m_hp[idx], r_cutoff, inclusive=False
        )
dir_m_hp = clusters.enforce_single_patch_dir_contact(dir_m_hp_dirty)
dir_m_hc = clusters.generate_mon_bind_direct(dir_m_hp, 2)
dir_m_hc_dangler, dir_m_hc_bridger = \
    clusters.split_binder_matrix(dir_m_hc)
dir_hc_hc = clusters.find_binder_clusters(dir_m_hc)
bonds_stat = clusters.count_foci_bonds(dir_hc_hc)
bonds_t[idx] = bonds_stat
full_hc_hc = clusters.generate_contact_matrix(dir_hc_hc)
print(full_hc_hc)
if not clusters.is_symmetric(full_hc_hc):
    print("full")
    print(idx)
clusters_stat = clusters.count_foci_clusters(full_hc_hc)

In [ ]:
for col in range(dir_m_hc.shape[1]):
    #print(np.nonzero(dir_m_hp_dirty[:,col])[0])
    #print(dir_m_hp[:,col])
    print(col)
    print("patch 1: ", np.nonzero(dir_m_hp[:,2*col])[0],"patch 2: ", np.nonzero(dir_m_hp[:,2*col+1])[0], "whole: ", np.nonzero(dir_m_hc[:,col])[0])
    print("dangle: ", np.nonzero(dir_m_hc_dangler[:,col])[0], "bridge: ", np.nonzero(dir_m_hc_bridger[:,col])[0])
    print("hn_hc_ dir: ", np.nonzero(dir_hc_hc[:,col])[0])

In [ ]:
for col in range(dir_m_hc.shape[1]):
    #print(np.nonzero(dir_m_hp_dirty[:,col])[0])
    #print(dir_m_hp[:,col])
    print(col)
    print("hn_hc_dir: ", np.nonzero(dir_hc_hc[:,col])[0])
    print("hn_hc_full: ", np.nonzero(full_hc_hc[:,col])[0])
    

In [ ]:
for col in range(dir_m_hp.shape[1]):
    #print(dir_m_hp[:,col])
    print(np.nonzero(dir_m_hp[:,col])[0])

In [ ]:
print((full_hc_hc-full_hc_hc.T)[1])
print((full_hc_hc-full_hc_hc.T)[6])
print((full_hc_hc-full_hc_hc.T)[9])
print((full_hc_hc-full_hc_hc.T)[24])
print((full_hc_hc-full_hc_hc.T).sum(axis=1))
print((full_hc_hc-full_hc_hc.T).sum(axis=0))

## Prober

In [ ]:
test = np.loadtxt("./N200kbmm2nh16ac2l25epshc0nc0ens1.ring-nucleoid3349-direct_contact.txt", dtype=int)
#print(test)
_, n_cols = test.shape
#print(n_cols)
contact_matrix = test.copy()
# Initialize a variable to keep track of the starting index for
# combinations
start_idx = 0
# Use a while loop to allow resetting the start index when necessary
while start_idx < n_cols:
    print("start_idx:", start_idx)
    # No update in this iteration initially
    updated = False
    # Loop over pairs of column indices starting from start_idx
    for i,j in combinations(range(start_idx, n_cols),2):
        if np.any(contact_matrix[:, i] & contact_matrix[:, j]) \
        and not np.array_equal(contact_matrix[:, i], contact_matrix[:, j]):
            print("i:", i)
            print("j:", j)
            #if np.array_equal(contact_matrix[:, i], transient) and np.array_equal(contact_matrix[:, j], transient):
            transient = contact_matrix[:, i] | contact_matrix[:, j]
            contact_matrix[:, i] = transient
            contact_matrix[:, j] = transient
            # Update the start index to the lower of the two indices
            start_idx = min(i, start_idx)
            updated = True
            # Break out of the for-loop to restart from updated start_idx
            break
    # If no updates were made in this iteration, increment start_idx
    if updated is False:
        start_idx += 1

print(contact_matrix)

In [2]:
# use this file to probe trj bug wholes in cubic geometry
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober

# analyzing bug files.
group = 'nucleoid'
lineage = 'whole'
save_to = '/Users/amirhsi_mini/research_data/' 
database = '/Users/amirhsi_mini/research_data/HnsCub/N*'
bug_pairs = glob(database + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
bug_pairs

[('/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh4ac2l25epshc1nc0ens1.ring.nucleoid.data',
  '/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh4ac2l25epshc1nc0ens1.ring.nucleoid.lammpstrj'),
 ('/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh4ac2l25epshc1nc11937ens1.ring.nucleoid.data',
  '/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh4ac2l25epshc1nc11937ens1.ring.nucleoid.lammpstrj'),
 ('/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh16ac2l25epshc0nc0ens1.ring.nucleoid.data',
  '/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh16ac2l25epshc0nc0ens1.ring.nucleoid.lammpstrj'),
 ('/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh16ac2l25epshc0nc11937ens1.ring.nucleoid.data',
  '/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh16ac2l25epshc0nc11937ens1.ring.nucleoid.lammpstrj'),
 ('/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh30ac2l25epshc0nc0ens1.ring.nucleoid.data',
  '/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh30ac2l25epshc0nc0ens1.ring.

In [4]:
for (bug_topo, bug_trj) in bug_pairs[2:-5]:
    print(bug_trj)
    prober.hns_nucleoid_cub_dis_hc_hc_cluster(bug_topo, bug_trj, lineage, save_to=save_to)

/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh16ac2l25epshc0nc0ens1.ring.nucleoid.lammpstrj
Setting the name of analyze file...

N200kbmm2nh16ac2l25epshc0nc0ens1.ring-nucleoid is analyzing...

done.
/Users/amirhsi_mini/research_data/HnsCub/N200kbmm2nh16ac2l25epshc0nc11937ens1.ring.nucleoid.lammpstrj
Setting the name of analyze file...

N200kbmm2nh16ac2l25epshc0nc11937ens1.ring-nucleoid is analyzing...

done.


# Plotting

## Mon-HNS-Patch-Distance

In [ ]:
database = '/Users/amirhsi_mini/research_data/hns_clusters/N*/N*/'
suffices = ["bondsHistBridgeTHnsCore", "bondsHistDangleTHnsCore", 
            "bondsHistTHnsCore", "clustersHistBridgeTHnsCore","clustersHistDangleTHnsCore","clustersHistTHnsCore"]
#suffices = ["bondsHistTHnsCore", "clustersHistTHnsCore"]
wholes = organizer.sort_filenames(glob(database+"N*error_logs.csv"),fmts=['error_logs.csv']) 

In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')
dfs = []
for f in wholes:
    #print(f)
    df = pd.read_csv(f[0])
    sim_info = HnsCub(
                f[0],
                'whole',
                'cubic',
                'nucleoid',
                'ring'
            )
    if not df.empty:
        #print(sim_info.whole)
        df["whole"] = sim_info.whole
        df["nhns"] =  sim_info.nhns
        df["phi_c_bulk_round"] = round(sim_info.phi_c_bulk,3)
        df["eps_hc"] = sim_info.eps_hc
        df["ens"] = sim_info.ensemble_id
        dfs.append(df)

In [ ]:
dfs = pd.concat(dfs)

In [ ]:
dfs.groupby(['nhns','phi_c_bulk_round', 'eps_hc', "ens"]).size().reset_index(name='counts')

In [ ]:

database = '/Users/amirhsi_mini/research_data/'
#suffices = ["bondsHistBridgeTHnsCore", "bondsHistDangleTHnsCore", 
#            "bondsHistTHnsCore", "clustersHistBridgeTHnsCore","clustersHistDangleTHnsCore","clustersHistTHnsCore"]
suffices = ["bondsHistTHnsCore", "clustersHistTHnsCore"]
df = defaultdict(list)
for suffix in suffices:
    wholes = organizer.sort_filenames(glob(database+"N*.npy"),fmts=[suffix+'.npy']) 
    for whole in wholes:
        sim_info = HnsCub(
                whole[0],
                'whole',
                'cubic',
                'nucleoid',
                'ring'
            )
        whole_array = np.load(whole[0])
        whole_mean = np.mean(whole_array,axis=0)
        for idx in range(len(whole_mean)):
            df["bin_center"].append(idx)
            df["value"].append(whole_mean[idx])
            df["property"].append(suffix)
            df["whole"].append(sim_info.whole)
            df["nhns"].append(sim_info.nhns)
            df["phi_c_bulk_round"].append(round(sim_info.phi_c_bulk,3))
            df["eps_hc"].append(sim_info.eps_hc)
        
df = pd.DataFrame.from_dict(df)   

In [ ]:
a = np.load(wholes[0][0])
a.sum(axis=0)/ a.shape[0]

In [ ]:
database = '/Users/amirhsi_mini/research_data/'
#suffices = ["bondsHistBridgeTHnsCore", "bondsHistDangleTHnsCore", 
#            "bondsHistTHnsCore", "clustersHistBridgeTHnsCore","clustersHistDangleTHnsCore","clustersHistTHnsCore"]
suffices = ["bondsHistTHnsCore", "clustersHistTHnsCore"]
df = defaultdict(list)
for suffix in suffices:
    wholes = organizer.sort_filenames(glob(database+"N*.npy"),fmts=[suffix+'.npy']) 
    for whole in wholes:
        sim_info = HnsCub(
                whole[0],
                'whole',
                'cubic',
                'nucleoid',
                'ring'
            )
        whole_array = np.load(whole[0])
        whole_mean = np.mean(whole_array,axis=0)
        for idx in range(len(whole_mean)):
            df["bin_center"].append(idx)
            df["value"].append(whole_mean[idx])
            df["property"].append(suffix)
            df["whole"].append(sim_info.whole)
            df["nhns"].append(sim_info.nhns)
            df["phi_c_bulk_round"].append(round(sim_info.phi_c_bulk,3))
            df["eps_hc"].append(sim_info.eps_hc)
        
df = pd.DataFrame.from_dict(df)   

In [ ]:
df.loc[(df["phi_c_bulk_round"]==0.4)&(df["bin_center"].isin([1,2,3,4,5]))].groupby(["property","bin_center"])["value"].max()

In [ ]:
sns.set_theme(
    style='ticks',
    rc={'axes.grid': True}
)
grid = sns.relplot(
    data=df,
    x="bin_center",
    y="value",
    col="property",
    col_wrap=3,
    hue="phi_c_bulk_round",
    legend="full",
    kind="line",
    marker="s",
    height=4,
    aspect=1
)
grid.savefig("../../test_plots/cluster.pdf")
plt.close()

## Probes

In [19]:

database = '/Users/amirhsi_mini/research_data/'
#suffices = ["bondsHistBridgeTHnsCore", "bondsHistDangleTHnsCore", 
#            "bondsHistTHnsCore", "clustersHistBridgeTHnsCore","clustersHistDangleTHnsCore","clustersHistTHnsCore"]
suffices = ["bondsHistDirDepTHnsCore", "clustersHistDirDepTHnsCore"]
df = defaultdict(list)
for suffix in suffices:
    wholes = organizer.sort_filenames(glob(database+"N*.npy"),fmts=[suffix+'.npy']) 
    for whole in wholes:
        sim_info = HnsCub(
                whole[0],
                'whole',
                'cubic',
                'nucleoid',
                'ring'
            )
        whole_array = np.load(whole[0])
        whole_mean = np.mean(whole_array,axis=0)
        for idx in range(len(whole_mean)):
            df["bin_center"].append(idx)
            df["value"].append(whole_mean[idx])
            df["property"].append(suffix)
            df["whole"].append(sim_info.whole)
            df["nhns"].append(sim_info.nhns)
            df["phi_c_bulk_round"].append(round(sim_info.phi_c_bulk,3))
            df["eps_hc"].append(sim_info.eps_hc)
        
df = pd.DataFrame.from_dict(df)   

In [20]:
df

,bin_center,value,property,whole,nhns,phi_c_bulk_round,eps_hc
0,0,14.150592,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0
1,1,1.388131,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0
2,2,0.359232,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0
3,3,0.081996,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0
4,4,0.014949,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0
...,...,...,...,...,...,...,...
366,26,0.124494,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1
367,27,0.146643,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1
368,28,0.143593,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1
369,29,0.089996,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1


In [21]:
# drop cluster with bin_center ==0:
condition = (df['property'] == 'C') & (df['bin_center'] == 0)
df_clean = df[~condition]

In [22]:
df_clean

,bin_center,value,property,whole,nhns,phi_c_bulk_round,eps_hc
0,0,14.150592,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0
1,1,1.388131,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0
2,2,0.359232,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0
3,3,0.081996,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0
4,4,0.014949,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0
...,...,...,...,...,...,...,...
366,26,0.124494,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1
367,27,0.146643,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1
368,28,0.143593,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1
369,29,0.089996,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1


In [13]:
#max_values = df_clean.groupby(['property','nhns','phi_c_bulk_round','eps_hc'])['value'].transform('max')
# Normalize 'value' by the maximum 'value' in each group
#df_clean['normalized_value'] = df_clean['value'] / max_values


#grouped = df.groupby(['property', 'nhns'])

# Normalize 'value' by 'nhns' within each group
df['normalized_value'] = df_clean.groupby(['property','nhns','phi_c_bulk_round','eps_hc'])['value'].transform(lambda x: x / x.name[1])

# Now df has an additional column 'normalized_value' with the normalized values


# Now df has an additional column 'normalized_value' with the normalized values


In [14]:
df

,bin_center,value,property,whole,nhns,phi_c_bulk_round,eps_hc,normalized_value
0,0,14.150592,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0,0.884412
1,1,1.388131,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0,0.086758
2,2,0.359232,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0,0.022452
3,3,0.081996,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0,0.005125
4,4,0.014949,bondsHistDirDepTHnsCore,N200kbmm2nh16ac2l25epshc0nc0ens1.ring,16,0.0,0,0.000934
...,...,...,...,...,...,...,...,...
366,26,0.124494,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1,0.004150
367,27,0.146643,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1,0.004888
368,28,0.143593,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1,0.004786
369,29,0.089996,clustersHistDirDepTHnsCore,N200kbmm2nh30ac2l25epshc1nc11937ens1.ring,30,0.4,1,0.003000


In [24]:
sns.set_theme(
    style='ticks',
    rc={'axes.grid': True}
)
grid = sns.relplot(
    data=df,
    x="bin_center",
    y="value",
    col="property",
    hue="phi_c_bulk_round",
    style="nhns",
    size="nhns",
    row="eps_hc",
    legend="full",
    kind="line",
    marker="s",
    height=4,
    aspect=1.5
)
grid.tight_layout()
grid.savefig("../../test_plots/cluster.pdf")
plt.close()